In [206]:
import time
import re

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="My-app")
location = geolocator.geocode("VIALE DELLA VITTORIA N. 217 92100")

tmp=re.search('(\w*), Sicilia', location.raw['display_name'],2)
tmp[1]

'Agrigento'

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from pathlib import Path
import re
import pandas as pd

df_private = pd.read_csv("../datasets/disgrazia.csv", delimiter=";")

df_private.columns = df_private.columns.str.strip()

geolocator = Nominatim(user_agent="andreaspin01@hotmail.it")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1) #, min_delay_seconds=0.1

LOC = []
LAT = []
LON = []

df_private = df_private.astype({'Indirizzo':str,'Citta\'':str})

for via, citta in zip(df_private["Indirizzo"], df_private["Citta\'"]):
    via = via.lower().replace('a\'', 'a').replace('e\'', 'e').replace('i\'', 'i').replace('o\'', 'o').replace('u\'', 'u')
    citta = citta.lower().replace('a\'', 'a').replace('e\'', 'e').replace('i\'', 'i').replace('o\'', 'o').replace('u\'', 'u')

    location = geocode(via + " " + citta)

    if(location is not None):
        cap = re.search(r'\d{5}',location.address)

        if cap is not None:
            LOC.append(cap.group())
        else:
            LOC.append(00000)

        if location.latitude is not None:
            LAT.append(location.latitude)
        else:
            LAT.append(0.0)

        if location.longitude is not None:
            LON.append(location.longitude)
        else:
            LON.append(0.0)
            
df_private.insert(loc=0, column='CAP', value=pd.Series(LOC))
df_private.insert(loc=0, column='LONGITUDINE', value=pd.Series(LON))
df_private.insert(loc=0, column='LATITUDINE', value=pd.Series(LAT))

filepath = Path('../datasets/private3.csv')
df_private.to_csv(filepath, index=False)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from pathlib import Path
import re

geolocator = Nominatim(user_agent="stv_project")
geocode = RateLimiter(geolocator.geocode) #, min_delay_seconds=0.1

COM = []
LAT = []
LON = []

for via, cap in zip(df_pubbliche["Indirizzo"], df_pubbliche["CAP"]):

    location = geocode(via + " " + str(cap))

    if(location is not None):
        comune = re.search(r'(\w*), Sicilia', location.address)

        if comune is not None:
            COM.append(comune[1])
        else:
            COM.append('######')

        if location.latitude is not None:
            LAT.append(location.latitude)
        else:
            LAT.append(0.0)

        if location.longitude is not None:
            LON.append(location.longitude)
        else:
            LON.append(0.0)
            
df_pubbliche.insert(loc=0, column='COMUNE', value=pd.Series(COM))
df_pubbliche.insert(loc=0, column='LONGITUDINE', value=pd.Series(LON))
df_pubbliche.insert(loc=0, column='LATITUDINE', value=pd.Series(LAT))

filepath = Path('../datasets/pubbliche.csv')
df_pubbliche.to_csv(filepath, index=False)